In [1]:
## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hii'
# | Hi?2004l

## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [2]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [3]:
alias | wc -l

0


In [4]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1210)
## *** All output from one run to the next needs to be the same ***
# Setting a temp directory for tests
TMP=/tmp/test-calc

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1210

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-calc/test-1210


In [5]:
# Count aliases proper
alias | wc -l

1


In [6]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

0


In [7]:
alias testnumhex="sed -r "s/[0-9,A-F,a-f]/X/g"" 
alias testuser="sed -r "s/"$USER"+/user/g"" 

In [8]:
## THE CORE OF CALCULATOR COMMAND FUNCTIONS

function old-calc () { echo "$@" | bc -l; }
# EX: perl-calc "2 / 4" => 0.500

## REPLACED perl- wth perl (BEFORE perlcalc.perl) & ADDED $BIN_DIR/ DUE TO COMMAND NOT FOUND ERROR
function perl-calc () { perl $BIN_DIR/perlcalc.perl -args "$@"; }
# TODO: read up on variable expansion in function environments
function perl-calc-init () { initexpr="$1"; shift; echo "$@" | perl perlcalc.perl -init="$initexpr" -; }
alias calc='perl-calc'
alias calc-prec6='perl-calc -precision=6'
alias calc-init='perl-calc-init'
alias calc-int='perl-calc -integer'
# TODO: use '$@' for '$*' (or note why not appropriate)
function old-perl-calc () { perl -e "print $*;"; }
function hex2dec { perl -e "printf '%d', 0x$1;" -e 'print "\n";'; }
function dec2hex { perl -e "printf '%x', $1;" -e 'print "\n";'; }
function bin2dec { perl -e "printf '%d', 0b$1;" -e 'print "\n";'; }
function dec2bin { perl -e "printf '%b', $1;" -e 'print "\n";'; }

alias hv='$BIN_DIR/hexview.perl'

In [9]:
## TEST 1 - old-calc SEEMS TO WORK PRETTY WELL
old-calc "24 / 4.0 - (35 / 7.0) * 5" # returns -19 as float
linebr
old-calc "(2^3)*(2^2)" # returns 32
linebr
old-calc "5*4*3*2*1+1" # returns 121

-19.00000000000000000000
--------------------------------------------------------------------------------
32
--------------------------------------------------------------------------------
121


In [10]:
## TEST 2 - old-perl-calc WORKS TOO
old-perl-calc "70 + 69 + 68"
printf "\n"
linebr
old-perl-calc "8/8/8/8/8"
printf "\n"
linebr

207
--------------------------------------------------------------------------------
0.001953125
--------------------------------------------------------------------------------


In [11]:
## TEST 3A - hex2dec
hex2dec "42Df5D144" 
#cases don't matter in the number
linebr
hex2dec "F"

17950953796
--------------------------------------------------------------------------------
15


In [12]:
## TEST 3B - hex2dec
dec2hex "128"
#inclusion of non-decimal numbers yield 0 as a result
dec2hex "A12" 

80
0


In [13]:
## TEST 3C - bin2dec
bin2dec "11110011"
linebr
#inclusion of non-binary numbers leads to error
bin2dec "0110" 

243
--------------------------------------------------------------------------------
6


In [14]:
## TEST 3D - dec2bin
dec2bin "70419"
#inclusion of non-decimal number leads to error
dec2bin "10"

10001001100010011
1010


In [15]:
## TEST 4 - hv
# creates hexview of ping.sh file
pwd
rm -rf ./*
# $ hv "ping.sh"
# | Can't open ping.sh: No such file or directory
ps -l > testforhv.txt
hv testforhv.txt | testnumhex

/tmp/test-calc/test-1210
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  X S   UIX     PI
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  X    PPIX  X PRI
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX    NI XXXR SZ WXH
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  XN  TTY         
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX   TIMX XMX.X S  X
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  XXX    XXXX    X
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  XXX  X  XX   X -
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX    XXXX Xo_wXi pt
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  s/X    XX:XX:XX 
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  XXsh.X R  XXXX  
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX    XXXX    XXXX  
XXXXXXXX  XX XX XX XX XX XX XX XX - XX XX XX XX XX XX XX XX  X  XX   X -  XXX
XXXXXXXX  XX XX XX XX XX XX XX XX - XX 

In [16]:
## TEST 5 - calc AND ITS ALIASES

## ERROR IN CODE 

# $ calc "9+10"
# | Number found where operator expected at (eval 34) line 1, near "args 9"
# |         (Do you need to predeclare args?)
# | Missing argument in printf at /home/xaea12/bin/perlcalc.perl line 117, <> line 1.
# | 0.000

# $ echo "12+3" > calculateit.txt
# $ perl-calc "2 / 4"

In [17]:
## ERROR

# $ perl-calc-init "9+10"
# | Missing argument in printf at /home/xaea12/bin/perlcalc.perl line 117, <> line 1.
# | 0.000 